In [2]:
import os

# Query twitter API

In [67]:
import os
import requests
# from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv, find_dotenv


search_url = "https://api.twitter.com/2/tweets/search/recent"
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

def get_token():
    env_path = os.path.join(os.path.dirname(os.getcwd()),'.env')
    env_path = find_dotenv() # automatic find 
    load_dotenv(env_path)
    token = os.getenv('TWITTER_BEARER_TOKEN','Key missing in env settings')
    return token

def last_date_db():
    """
    Method to retrieve the last date in the DB
    """
    # Construct a BigQuery client object.

    query = """
        SELECT created_at
        FROM `wagon-bootcamp-802.my_dataset.new_table`
        ORDER BY created_at DESC
        LIMIT 1
    """
    df = pd.read_gbq(query, dialect='standard')
    return df.iloc[0]['created_at'].tz_localize(None).isoformat()+'Z'


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    bearer_token = get_token()
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def query_twitter(start_time=None, next_token=None):
    query_params = {'query': '#F1',
                    'tweet.fields':'created_at,lang',
                    'start_time':start_time,
                    'max_results' : '100',
                    'next_token': next_token
                   }
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response


def func():
    '''
    Pushing results to GBQ
    '''
    most_recent_dt = last_date_db()
    response = query_twitter(most_recent_dt)
    data = pd.DataFrame()
    while response['meta'].get('next_token',False):
        df = pd.DataFrame(response['data'])[['text', 'created_at', 'id', 'lang']]
        df = df[df['lang'] == 'en']
        df['created_at'] = pd.to_datetime(df['created_at'])
        data = data.append(df, ignore_index=True)
        response = query_twitter(most_recent_dt,response['meta'].get('next_token',False))
    df = pd.DataFrame(response['data'])[['text', 'created_at', 'id', 'lang']].iloc[:-1]
    df = df[df['lang'] == 'en']
    df['created_at'] = pd.to_datetime(df['created_at'])
    data = data.append(df, ignore_index=True)
    table_id = 'wagon-bootcamp-802.my_dataset.new_table'
    data.to_gbq(table_id, if_exists='append')
    return df

In [68]:
func()

/home/jeremymarchand/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 

In [65]:
most_recent_dt = last_date_db()
response = query_twitter(most_recent_dt)
data = pd.DataFrame()
while response['meta'].get('next_token',False):
    df = pd.DataFrame(response['data'])[['text', 'created_at', 'id', 'lang']]
    df = df[df['lang'] == 'en']
    df['created_at'] = pd.to_datetime(df['created_at'])
    data = data.append(df, ignore_index=True)
    response = query_twitter(most_recent_dt,response['meta'].get('next_token',False))
df = pd.DataFrame(response['data'])[['text', 'created_at', 'id', 'lang']].iloc[:-1]
df = df[df['lang'] == 'en']
df['created_at'] = pd.to_datetime(df['created_at'])
data = data.append(df, ignore_index=True)

/home/jeremymarchand/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [66]:
data

,text,created_at,id,lang
0,#F3 - 1st win of the season\n#F2 - 1st win of ...,2022-07-03 10:36:42+00:00,1543544252122136576,en
1,RT @F1: Could the rain have a part to play in ...,2022-07-03 10:36:37+00:00,1543544234078248962,en
2,RT @NataliePinkham: Please don’t boo. Pass it ...,2022-07-03 10:36:37+00:00,1543544232949997568,en
3,RT @F1: Could the rain have a part to play in ...,2022-07-03 10:36:36+00:00,1543544228021768192,en
4,RT @F1: IT’S A FIRST POLE FOR CARLOS SAINZ!!! ...,2022-07-03 10:36:36+00:00,1543544227115720704,en
...,...,...,...,...
2235,RT @fiagirly: #F1: During investigations on th...,2022-07-03 09:09:58+00:00,1543522425190490113,en
2236,RT @fiagirly: #F1: Toto Wolff on finding out s...,2022-07-03 09:09:57+00:00,1543522421411561473,en
2237,"RT @5ireChain: At 5ire, we believe in unity an...",2022-07-03 09:09:56+00:00,1543522418660089856,en
2238,RT @F1: A breathless finale to qualifying on S...,2022-07-03 09:09:56+00:00,1543522418064592897,en


In [64]:
df.columns

Index(['created_at', 'text', 'id', 'lang'], dtype='object')